In [ ]:
import numpy as np 
import pandas as pd
import os

from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier
from sklearn.metrics import mean_squared_error, accuracy_score

In [ ]:
train_data = pd.read_csv("../input/sept-tps-5fold-stratified/sept_TPS_train_5_folds.csv")
test_data = pd.read_csv("../input/tabular-playground-series-sep-2021/test.csv")
sample_solution = pd.read_csv("../input/tabular-playground-series-sep-2021/sample_solution.csv")

X_test = test_data.copy()
X_test = X_test.drop(['id'], axis=1)

In [ ]:
test_data.shape

In [ ]:
exclude_cols = ["id", "kfold", "claim"]
useful_cols = [i for i in train_data.columns if i not in exclude_cols]
feature_cols = [col for col in train_data.columns if col.startswith('f')]

In [ ]:
num_cols_with_missing = sum(train_data.isnull().sum() > 0)
num_cols_with_missing

In [ ]:
test_data.head()

In [ ]:
final_predictions = []
for fold in range(5):
    X_train = train_data[train_data.kfold != fold].reset_index(drop=True)
    X_valid = train_data[train_data.kfold == fold].reset_index(drop=True)
    
    my_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
    
    X_train[feature_cols] = pd.DataFrame(my_imputer.fit_transform(X_train[feature_cols]))
    X_valid[feature_cols] = pd.DataFrame(my_imputer.transform(X_valid[feature_cols]))
#     imputed_X_train.columns = X_train.columns
#     imputed_X_valid.columns = X_valid.columns
    
#     convert_dict = {'kfold': int,
#                 'claim': int
#                }
    
#     imputed_X_train = imputed_X_train.astype(convert_dict)
#     imputed_X_valid = imputed_X_valid.astype(convert_dict)
    
    y_train = X_train['claim']
    X_train = X_train.drop(exclude_cols, axis=1)
    y_valid = X_valid['claim']
    X_valid = X_valid.drop(exclude_cols, axis=1)
    
    X_test[feature_cols] = pd.DataFrame(my_imputer.transform(X_test[feature_cols]))
#     X_test.columns = test_data.columns
    
    model = XGBClassifier(objective="binary:logistic", random_state=fold, tree_method='gpu_hist', gpu_id=0, n_jobs=4)
    model.fit(X_train, y_train)
    preds_valid = model.predict(X_valid)
    test_preds = model.predict(X_test)
    final_predictions.append(test_preds)
    print(fold, accuracy_score(y_valid, preds_valid))

In [ ]:
test_data

In [ ]:
len(final_predictions[0])

In [ ]:
sample_solution['claim'] = final_predictions[0]

In [ ]:
sample_solution.head()

In [ ]:
sample_solution.to_csv("submission.csv", index=False)